In [1]:
from __future__ import print_function

# Jupyter display
from IPython.display import display

# json
import json

# widgets
import ipywidgets as widgets
import bqplot as bq
import ipyleaflet as ll

# numerics
import pandas as pd
import numpy as np

# colormap
import matplotlib as mpl
import matplotlib.cm
import matplotlib.colors

def n_colors(n, colormap=mpl.cm.Blues):
    data = np.linspace(0.0,1.0,n)
    c = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(data)]
    return c

def data_to_colors(data, colormap=mpl.cm.plasma):
    c = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(mpl.colors.Normalize()(data))]
    return c

# Satisfaction Score by ZIP Code

In [2]:
survey_map = ll.Map(center=[37.7749, -122.34580993652344], zoom=12, layout=widgets.Layout(height='450px'))

survey_data = pd.read_csv('./../analysis_data/scores_by_zipcodes.csv', header=None)
survey_colors = data_to_colors(survey_data.ix[:, 1], colormap=mpl.cm.Greens)
zips = [str(int(i)) for i in survey_data.ix[:, 0]]
cols = {}

for i in range(len(zips)):
    cols[zips[i]] = survey_colors[i]

# Survey layer
with open('./../mapdata/sf_zipcodes.geojson') as f:
    data = json.load(f)

for feature in data['features']:
    feature['properties']['style'] = {
        'color': cols[feature['id']],
        'weight': 1,
        'fillColor': cols[feature['id']],
        'fillOpacity': 0.65,
    }

survey_layer = ll.GeoJSON(data=data, hover_style={'fillColor': 'red'})
survey_map += survey_layer

with open('./../mapdata/bayarea_nosf_zipcodes.geojson') as f:
    bay_data = json.load(f)
    
# Grey layer
grey_colors = n_colors(len(bay_data['features']), colormap=mpl.cm.Greys)

for feature, color in zip(bay_data['features'], grey_colors):
    feature['properties']['style'] = {
        'color': 'gray',
        'weight': 1,
        'fillColor': 'gray',
        'fillOpacity': 0.2
    }

gray_layer = ll.GeoJSON(data=bay_data, hover_style={'fillColor': 'red'})

survey_map.add_layer(gray_layer)

survey_map

# Neighborhood features

In [3]:
feature_map = ll.Map(center=[37.7749, -122.34580993652344], zoom=12, layout=widgets.Layout(height='450px'))
tract_data = pd.read_csv('./../data/CensusData/sfo data/tract_data_normalized.csv')
colors = data_to_colors(tract_data['pct_bachelors'], colormap=mpl.cm.Reds)
tracts = [str(int(i)) for i in tract_data.ix[:, 0].values]

cols = {}
for i in range(len(tracts)):
    cols[tracts[i][4:]] = colors[i]

with open('./../mapdata/sf_tracts.geojson') as f:
    data = json.load(f)
for feature in data['features']:
    try:
        style_col = cols[feature['properties']['tractce10']]
    except KeyError:
        style_col = 'Grey'
    feature['properties']['style'] = {
        'color': style_col,
        'weight': 1,
        'fillColor': style_col,
        'fillOpacity': 0.75,
    }

feature_layer = ll.GeoJSON(data=data)
feature_map += feature_layer

data_columns = [
      'pct_bachelors',
      'labor_part_rate',
      'pct_welfare',
      'low_pov_idx',
      'labor_idx',
      'env_health_idx',
      'hispanic',
      'white',
      'black',
      'american_indian',
      'asian',
      'pac_islander',
      'other_races',
      'two_races',
      'housing_cost',
      'transportation_cost',
      'pub_school_score',
      'pr_school_score',
      'rest_score',
      'rest_proximity']

dpdown = widgets.Dropdown(options=data_columns, value='pct_bachelors')

def update_dpdown(change):
    value = change['new']
    colors = data_to_colors(tract_data[value], colormap=mpl.cm.Reds)
    cols = {}
    for i in range(len(tracts)):
        cols[tracts[i][4:]] = colors[i]
    for feature in data['features']:
        try:
            style_col = cols[feature['properties']['tractce10']]
        except KeyError:
            style_col = 'grey'
        feature['properties']['style'] = {
            'color': style_col,
            'weight': 1,
            'fillColor': style_col,
            'fillOpacity': 0.75,
        }
        
    feature_layer = ll.GeoJSON(data=data)
    
    feature_map.layers = [feature_map.layers[0], feature_layer]
    
dpdown.observe(update_dpdown, names=['value'])

feature_map.layout.align_self = 'stretch'
display(widgets.VBox([feature_map, dpdown]))

# From the Features to the Satisfaction Index

In [11]:
ordinal_features_scale = bq.OrdinalScale()
ord_axis = bq.Axis(scale=ordinal_features_scale)
values_features_scale = bq.LinearScale()
value_axis = bq.Axis(scale=values_features_scale, orientation='vertical')

scores_zipcode = pd.read_csv('./../analysis_data/scores_by_zipcodes.csv')

bars = bq.Bars(x=data_columns, y=np.zeros(len(data_columns)), 
               scales={
        'x': ordinal_features_scale,
        'y': values_features_scale
    })

figure = bq.Figure(axes=[value_axis, ord_axis], marks=[bars], animate_duration=500)
figure

In [12]:
scores_zipcode

,94101,8.28188818172
0,94102,7.872967
1,94103,8.022743
2,94104,7.649829
3,94105,7.310335
4,94106,8.399158
5,94107,7.975440
6,94108,8.473097
7,94109,7.644697
8,94110,8.403898
9,94111,8.370478


In [8]:
ls ../analysis_data/scores_by_zipcodes.csv

311_Information_Requests_by_Month.csv  crime_data_8.csv
AHBP_eligible_parcels.csv              crime_data_9.csv
Untitled.ipynb                         crime_part_1.csv
Untitled1.ipynb                        data read me.docx
aff_index.csv                          demographics_sfo_2010 census.csv
census_data_plus_affordability.csv     google_transit/
census_tract_numbers_for_sfo.xls       living_affordability_index_data.csv
census_tracts_to_loc_list.txt          race_data_2010.csv
crime_data_0.csv                       tract_census_data.csv
crime_data_1.csv                       tract_census_data_new.csv
crime_data_2.csv                       tract_coordinates.csv
crime_data_3.csv                       tract_data_normalized.csv
crime_data_4.csv                       tract_data_with_schools.csv
crime_data_5.csv                       tracts_reduced_data.csv
crime_data_6.csv                       ~$ta read me.docx
crime_data_7.csv                       ~WRL2499.tmp
